# What is a neural network
ニューラルネットワークは、層によって接続されたニューロンの集合体である。各ニューロンは小さな計算ユニットで、簡単な計算を行い、問題をまとめて解決します。それらは層で構成されています。層には、入力層、隠れ層、出力層の3種類があります。各層は、入力層を除いて、多数のニューロンを含んでいます。ニューラルネットワークは、人間の脳が情報を処理する方法を模倣しています。

以下のセクションでは、FashionMNISTデータセットの画像を分類するためのニューラルネットワークを構築します。

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Get a hardware device for training
GPUのようなハードウェアアクセラレータが利用できるのであれば、その上でモデルをトレーニングできるようにしたい。torch.cudaが利用可能かどうかを確認しましょう。そうでない場合は、引き続きCPUを使用します。

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


# Define the class
nn.Moduleをサブクラス化してニューラルネットワークを定義し、__init__でニューラルネットワークのレイヤーを初期化します。nn.Moduleのサブクラスはすべて、forwardメソッドで入力データに対する操作を実装しています。

我々のニューラルネットワークは以下のように構成されている：

- 入力層は28x28、つまり784の特徴量/画素を持つ。
- 最初の線形モジュールは、入力された784個の特徴量を受け取り、それを512個の特徴量を持つ隠れ層に変換します。
- 変換の際にはReLU活性化関数が適用される
- 2番目の線形モジュールは、最初の隠れ層から512個の特徴を入力として受け取り、512個の特徴を持つ次の隠れ層へ変換します。
- 変換の際にはReLU活性化関数が適用されます。
- 第3の線形モジュールは、第2隠れ層から512個の特徴を入力とし、クラス数である10で出力層に変換します。
- 変換の際にはReLU活性化関数が適用されます。

Our neural network are composed of the following:
- The input layer with 28x28 or 784 features/pixels.
- The first linear module takes the input 784 features and transforms it to a hidden layer with 512 features
- The ReLU activation function will be applied in the transformation
- The second linear module take 512 features as input from the first hidden layer and transforms it to the next hidden layer with 512 features
- The ReLU activation function will be applied in the transformation
- The third linear module take 512 features as input from the second hidden layer and transforms it to the output layer with 10, which is the number of classes
- The ReLU activation function will be applied in the transformation

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

NeuralNetworkのインスタンスを作成し、デバイスに移動させ、構造を出力する

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


モデルを使うには、入力データを渡します。これにより、モデルのフォワードが実行され、いくつかのバックグラウンド操作が行われます。ただし、model.forward()を直接呼び出さないでください！入力に対してモデルを呼び出すと、各クラスの生の予測値を持つ10次元のテンソルが返されます。

これをnn.Softmaxのインスタンスに渡すことで、予測密度を得ることができます

In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X) 
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2])


# Weight and Bias
重みとバイアス
`nn.Linear`モジュールは、ランダムに初期化された.weightとbiasを各レイヤーごとに設定し、内部でTensorsに格納する。

In [6]:
print(f"First Linear weights: {model.linear_relu_stack[0].weight} \n")
print(f"First Linear weights: {model.linear_relu_stack[0].bias} \n")

First Linear weights: Parameter containing:
tensor([[-0.0178,  0.0106, -0.0004,  ..., -0.0083,  0.0151,  0.0128],
        [ 0.0346, -0.0181, -0.0210,  ...,  0.0189, -0.0036, -0.0006],
        [-0.0281, -0.0053,  0.0192,  ...,  0.0281,  0.0294, -0.0207],
        ...,
        [ 0.0342,  0.0275,  0.0175,  ..., -0.0156,  0.0035, -0.0066],
        [-0.0254,  0.0015,  0.0251,  ..., -0.0103, -0.0045, -0.0072],
        [ 0.0168, -0.0042,  0.0031,  ..., -0.0017,  0.0246,  0.0091]],
       requires_grad=True) 

First Linear weights: Parameter containing:
tensor([-0.0183, -0.0312,  0.0027, -0.0054, -0.0106,  0.0022, -0.0254, -0.0141,
         0.0042, -0.0352, -0.0043, -0.0236, -0.0087,  0.0156,  0.0033,  0.0039,
         0.0045, -0.0201,  0.0054, -0.0227, -0.0018, -0.0292,  0.0339, -0.0216,
        -0.0197,  0.0143,  0.0016,  0.0207, -0.0210,  0.0160,  0.0142, -0.0157,
        -0.0128,  0.0101, -0.0182, -0.0090,  0.0199,  0.0353, -0.0063, -0.0197,
        -0.0279,  0.0087, -0.0252,  0.0141, -0.00

# Model layers
FashionMNISTモデルのレイヤーを分解してみましょう。それを説明するために、サイズ28x28の3枚の画像からなるミニバッチのサンプルを取り、それをネットワークに通すとどうなるかを見ていきます。

In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


# nn.Flatten
nn.Flatten層を初期化して、各2D 28x28画像を784個のピクセル値からなる連続した配列に変換します（ミニバッチの次元（dim=0）は維持されます）。各ピクセルは、ニューラルネットワークの入力層に渡されます。

In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


# nn.Linear
線形層は、保存されている重みとバイアスを使用して入力に線形変換を適用するモジュールです。入力層の各ピクセルのゲイズケール値は、計算のために隠れ層のニューロンに接続されます。変換に使われる計算式は

 ${{weight * input + bias}} $.

In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


# nn.ReLU
非線形活性化とは、モデルの入力と出力の間の複雑なマッピングを作成するものです。線形変換の後に適用して非線形性を導入することで、ニューラルネットワークが様々な現象を学習するのに役立ちます。このモデルでは、線形層の間にnn.ReLUを使用していますが、他にもモデルに非線形性を導入するための活性化関数が存在します。

ReLU活性化関数は、線形層の計算から出力を取り出し、負の値をゼロに置き換えます。

Linear output: ${ x = {weight * input + bias}} $.  
ReLU:  $f(x)= 
\begin{cases}
    0, & \text{if } x < 0\\
    x, & \text{if } x\geq 0\\
\end{cases}
$

In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.1597, -0.0976, -0.1879, -0.1390, -0.2916, -0.0340, -0.0307, -0.0887,
          0.2640,  0.6074,  0.1367,  0.0716,  0.3189,  0.5606, -0.4308, -0.6087,
         -0.3813,  0.1452, -0.4784,  0.1390],
        [-0.1767, -0.2061,  0.0837, -0.1397, -0.2316, -0.2682, -0.0190,  0.1268,
          0.4273,  0.8158,  0.1763,  0.4070,  0.4186,  0.4841, -0.2449, -0.8118,
         -0.6087, -0.1698, -0.2118, -0.0985],
        [ 0.0628, -0.1324, -0.1085,  0.0373, -0.1707, -0.2249, -0.0337, -0.2461,
          0.4387,  0.5656,  0.1435, -0.0477,  0.2048,  0.5002, -0.2580, -1.0221,
         -0.1040, -0.1856, -0.8341,  0.0564]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2640,
         0.6074, 0.1367, 0.0716, 0.3189, 0.5606, 0.0000, 0.0000, 0.0000, 0.1452,
         0.0000, 0.1390],
        [0.0000, 0.0000, 0.0837, 0.0000, 0.0000, 0.0000, 0.0000, 0.1268, 0.4273,
         0.8158, 0.1763, 0.4070, 0.4186, 0.4841, 0.00

# nn.Sequential
nn.Sequentialは、モジュールの順序付きコンテナです。データは定義されたのと同じ順序ですべてのモジュールに渡されます。シーケンシャルコンテナを使って、seq_modulesのように素早くネットワークを組むことができます。

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

# nn.Softmax
ニューラルネットワークの最後の線形層は、logits - [-infty, infty]の生値を返し、nn.Softmaxモジュールに渡される。Softmax活性化関数は、ニューラルネットワークからの出力の確率を計算するために使用されます。これは、ニューラルネットワークの出力層でのみ使用されます。dimパラメータは、結果の値の合計が1になる次元を示します。最も高い確率を持つノードが目的の出力を予測します。

In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

# Model parameter
ニューラルネットワークの多くの層はパラメータ化されており、学習中に最適化される重みとバイアスが関連付けられています。nn.Moduleをサブクラス化すると、モデルオブジェクト内で定義されたすべてのフィールドが自動的に追跡され、モデルのparameters()やnamed_parameters()メソッドを使用してすべてのパラメータにアクセスできるようになりま

この例では、各パラメーターに対して反復処理を行い、そのサイズと値のプレビューを表示しています。

In [13]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0178,  0.0106, -0.0004,  ..., -0.0083,  0.0151,  0.0128],
        [ 0.0346, -0.0181, -0.0210,  ...,  0.0189, -0.0036, -0.0006]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0183, -0.0312], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-4.0978e-02,  3.4463e-04,  1.8284e-02,  ..., -3.5857e-02,
          1.9479e-02, -9.7720e-03],
        [ 8.2441e-06,  1.0563e-02,  1.4524e-02,  ..., -8.0478e-03,
         -2.4539e-02, 